In [1]:
import pickle
import string
import math
import multiprocessing
from datetime import datetime

from collections import Counter

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import fasttext as ft
import fasttext.util

In [2]:
%load_ext tensorboard

In [3]:
tf.config.list_physical_devices('GPU'), tf.test.is_built_with_cuda()

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], True)

In [4]:
# Our dataset is around ~15mb
df = pd.read_csv('old_general.csv', header=None, usecols=[2], names=['chat'])
df.head(10)

,chat
0,Nigger in the house
1,Nigger
2,"""Yes"""
3,I have a hard on right now
4,same tbh
5,im actually jerking off rn
6,I'm fapping to cosplaying kids
7,same
8,con season is best season
9,true


In [5]:
UNK = 'xxunk'
# OOV included in encoder

def raw_text_preprocess(x):
    return x.lower().translate(str.maketrans('', '', string.punctuation))

df['chat'] = df['chat'].apply(raw_text_preprocess)

df.head(10)

,chat
0,nigger in the house
1,nigger
2,yes
3,i have a hard on right now
4,same tbh
5,im actually jerking off rn
6,im fapping to cosplaying kids
7,same
8,con season is best season
9,true


In [6]:
df['tkn_len'] = df['chat'].apply(lambda x: len(x.split(' ')))

_p = {i: np.percentile([l for l in df['tkn_len'].values], i) for i in [70, 80, 90, 95, 97, 99]}

_p # {70: 7.0, 80: 9.0, 90: 12.0, 95: 16.0, 97: 19.0, 99: 28.0}

{70: 7.0, 80: 9.0, 90: 12.0, 95: 16.0, 97: 19.0, 99: 28.0}

In [7]:
MAX_TOKENS = 20

BOS = 'xxbos'
EOS = 'xxeos'

def add_utility_tokens(r):
    content = r['chat']

    return f'{BOS} {content} {EOS}'

df['chat'] = df.apply(add_utility_tokens, axis=1)

MAX_TOKENS += 2

df.head(10)

,chat,tkn_len
0,xxbos nigger in the house xxeos,4
1,xxbos nigger xxeos,1
2,xxbos yes xxeos,1
3,xxbos i have a hard on right now xxeos,7
4,xxbos same tbh xxeos,2
5,xxbos im actually jerking off rn xxeos,5
6,xxbos im fapping to cosplaying kids xxeos,5
7,xxbos same xxeos,1
8,xxbos con season is best season xxeos,5
9,xxbos true xxeos,1


In [8]:
df['chat_shift'] = df['chat'].shift(-1)
df.drop(df.tail(1).index, inplace=True) # drop last row for NaN due to shift

In [9]:
# tokenize and build vocab
tokenizer = tfds.features.text.Tokenizer()

OCCURENCES_TRESHOLD = 2

vocab_counter = Counter()

vocab_counter.update([BOS, EOS, UNK] * OCCURENCES_TRESHOLD)

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
  xb = row['chat']
  xb_tokens = tokenizer.tokenize(xb)
  vocab_counter.update(xb_tokens)

# keep only words that appear more than OCCURENCES_TRESHOLD
vocabulary_set = set(el for el in vocab_counter.elements() if vocab_counter[el] >= OCCURENCES_TRESHOLD)

vocabulary_set.update([BOS, EOS, UNK])
   
vocab_size = len(vocabulary_set)

encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, oov_token=UNK, lowercase=True) 

In [10]:
vocab_size

19922

In [11]:
# load fasttext model and build embeddings layer
# ft.util.download_model('en', if_exists='ignore')
en_vecs = ft.load_model('./cc.en.300.bin')
en_vecs.get_dimension() # 300

# Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.
# you can safely ignore this warning, it's for other model types and (possibly) deprecation warning for other versions

300

In [12]:
vocab_size = encoder.vocab_size # compensate for the 0 paddings

embedding_matrix = np.zeros((vocab_size, 300))

for word in vocabulary_set:
    i = encoder.encode(word)
    embedding_matrix[i] = en_vecs.get_word_vector(word)
    
vocab_size

19924

In [13]:
# release 8gb of memory by unloading the fasttext model
del en_vecs

In [14]:
ENCODER_INPUT_NAME = 'encoder_input'
DECODER_INPUT_NAME = 'decoder_input'
DECODER_OUTPUT_NAME = 'decoder_output'

X_ENC = []
X_DEC = []
Y = []

from tensorflow.keras.preprocessing.sequence import pad_sequences
# What if I don't want to use teacher forcing for training?
# > Custom training loop, look below:
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

# you could make this lazy-generator wise too
#for _, row in tqdm(df.iterrows(), total=df.shape[0]):

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    ch = row['chat']
    ch_shift = row['chat_shift']

    # print(f'{ch} - {ch_shift}')

    xb, yb = encoder.encode(ch), encoder.encode(ch_shift)
    if len(xb) >= MAX_TOKENS or len(yb) >= MAX_TOKENS:
        continue

    xb_padded = pad_sequences([xb], MAX_TOKENS, padding='post')

    # [enc_input, dec_input], dec_output
    for i in range(1, len(yb)-1):
        dec_in = pad_sequences([yb[0:i]], MAX_TOKENS, padding='post')
        dec_out = pad_sequences([yb[0:i+1]], MAX_TOKENS, padding='post')

        #yield {ENCODER_INPUT_NAME: xb_padded, DECODER_INPUT_NAME: dec_in}, {DECODER_OUTPUT_NAME: dec_out}
        
        # yields
        X_ENC.append(xb_padded) 
        X_DEC.append(dec_in)
        Y.append(dec_out)
        
X_ENC = np.squeeze(np.stack(X_ENC))
X_DEC = np.squeeze(np.stack(X_DEC))
Y = np.squeeze(np.stack(Y))

In [ ]:
type(X[0][0])

In [ ]:
# add corpus bleau score

In [68]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Reshape
# https://keras.io/examples/lstm_seq2seq/

# this one messes with layer names and appends :X numbers
tf.keras.backend.clear_session()

latent_dim = 128

input_tensor_type = tf.int64

encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix], name='encoder_embedding')
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix], name='decoder_embedding')

# input is (tokens, vocab size)
encoder_input = Input(shape=(MAX_TOKENS), dtype=input_tensor_type, name=ENCODER_INPUT_NAME)

# embedding dim is 300 from the fasttext model
encoder_embedding = encoder_embedding_layer(encoder_input) # None, MAX_TOKENS, 300

encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

encoder_states = [state_h, state_c]

# # same logic as model_input
decoder_input = Input(shape=( MAX_TOKENS,), dtype=input_tensor_type, name=DECODER_INPUT_NAME)
decoder_embedding = decoder_embedding_layer(decoder_input)

decoder_lstm_layer = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_lstm, _, _ = decoder_lstm_layer(decoder_embedding, initial_state=encoder_states)

decoder_dense_layer = Dense(vocab_size, activation='softmax', name=DECODER_OUTPUT_NAME)
decoder_output = decoder_dense_layer(decoder_lstm)

model = Model([encoder_input, decoder_input], decoder_output)

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 22)]         0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, 22)]         0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, 22, 300)      5977200     encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, 22, 300)      5977200     decoder_input[0][0]              
______________________________________________________________________________________________

In [16]:
# calculate how much unwrapping will the generator do
expanded_samples = 1062410
# expanded_samples = 0

# for _, row in tqdm(df.iterrows()):
#     ch = row['chat_shift']
#     e = encoder.encode(ch)
#     # print(e)
#     expanded_samples += len(e) - 2 # 1 for BOS, 1 for last token (EOS)
    
expanded_samples

1062410

In [17]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=3, monitor='loss')

In [18]:
## Training Logs:

# GTX 1060 6GB, i5-3570k (4 core 4 thread)

# LSTM Time Parameter = 22 (tokens)

# Latent dimension: 256: the model takes about ~ 30-40 min per epoch with batch size 64
# For batch size 256, vocab treshold 2, 10 epochs it repeats the same input over and over

# Latent dimension: 64, the model takes about ~ 30min per epoch with batch size 64
# for bs 64, vocab treshold 2 and 2 epochs, unk tokens are produced
# for 10 epochs, bs 64, slightly more meaningful text is produced, but with 
# for 20 epochs, bs 64, f

# latent dimension: 32, bs 64, epochs 20 (=> 20, patience 5), first token after xxbos is mostly 'x' and then unk

In [67]:
BATCH_SIZE = 256
EPOCHS = 5

history = model.fit([X_ENC, X_DEC], Y,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    #steps_per_epoch=expanded_samples//BATCH_SIZE,
                    callbacks=[tensorboard_callback, early_stop_callback])

Epoch 1/5
   64/13398 [..............................] - ETA: 43:58 - loss: 2.6495 - sparse_categorical_accuracy: 0.7465

KeyboardInterrupt: 

In [ ]:
encoder_model = Model(encoder_input, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_output, state_h, state_c = decoder_lstm_layer(
    decoder_embedding, 
    initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_output = decoder_dense_layer(decoder_output)

# reminder: decoder input -> decoder embedding (on the graph)
decoder_model = Model(
    [decoder_input] + decoder_states_inputs,
    [decoder_output] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
    enc_input_seq = encoder.encode(input_seq)
    padded = pad_sequences([enc_input_seq], MAX_TOKENS, padding='post')
    states_value = encoder_model.predict(padded)

    # Batch size is 1 this is why there is an extra sequence
    target_seq = np.zeros((1, MAX_TOKENS))
    # sampling recurrent loop

    i = 0
    target_seq[0, i] = encoder.encode(BOS)[0]
    # print(target_seq)

    decoded_sentence = [BOS]
    while True:
        i += 1
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        # print(output_tokens)
        
        # argmax the output to get next token
        sampled_token_index = np.argmax(output_tokens[0, i, :])
        
        sampled_word = UNK
        if sampled_token_index != 0:
            sampled_word = encoder.decode([sampled_token_index])[0]
            
        decoded_sentence += [sampled_word]

        # if max length or EOS, stop
        if (sampled_word == EOS or
           len(decoded_sentence) == MAX_TOKENS):
            break

        # update the target sequence (of length 1).
        # target_seq = np.zeros((1, 1, MAX_TOKENS))
        target_seq[0, i] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

# visualize some results from test data
test_data = df.sample(n=50, random_state=42)

for _, row in test_data.iterrows():
    ch = row['chat']
    ch_shift = row['chat_shift']
    
    print(f'Input sentence: {ch}')
    print(f'Expected sentence: {ch_shift}')
    decoded_sentence = decode_sequence(ch)
    print('-')
    print(f'Decoded sentence: {" ".join(decoded_sentence)}')
    print('')

In [ ]:
encoder.save_to_file('text_encoder')

In [19]:
model.save('full_0')
encoder_model.save('encoder_model_0')
decoder_model.save('decoder_model_0')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: full_0\assets


In [ ]:
# TODO: save embeddings for tensorboard inspection

In [104]:
%tensorboard --logdir logs/scalars

Launching TensorBoard...

KeyboardInterrupt: 

In [ ]:
# post training quantization